In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import udf, to_timestamp
import happybase
import datetime
import spacy
from spacy.lang.id import Indonesian
from spacy.lang.en import English
from nltk.corpus import stopwords
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql.types import ArrayType, StringType
import re
from nltk.stem import WordNetLemmatizer
import string
import datetime, time

In [2]:
spark = SparkSession.builder \
    .config("spark.mongodb.read.connection.uri", "mongodb://127.0.0.1/data_engineering.preprocessed.coll") \
    .config("spark.mongodb.write.connection.uri", "mongodb://127.0.0.1/data_engineering.preprocessed.coll") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0") \
    .getOrCreate()

24/04/27 15:10:26 WARN Utils: Your hostname, DESKTOP-R2270LN. resolves to a loopback address: 127.0.1.1; using 172.19.63.19 instead (on interface eth0)
24/04/27 15:10:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hdlmf/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hdlmf/.ivy2/cache
The jars for the packages stored in: /home/hdlmf/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ea4cc77d-261b-47d0-91b3-8e89321405c7;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.0 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 104ms :: artifacts dl 6ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   

In [3]:
connection = happybase.Connection('localhost', port=9090)

# Access the table
table = connection.table('redditTravel')

# Scan all rows in the table
print("Scanning all rows in the table:")
try:
    for key, data in table.scan():
        print(f"Row Key: {key.decode()}")
        for column, value in data.items():
            print(f"    {column.decode()}: {value.decode()}")
except Exception as e:
    print(f"Failed to read from table: {e}")

# Close the connection
connection.close()

Scanning all rows in the table:
Row Key: 0026476a20bfbd08714155bb66f0b4feb2d25c1c
    cf1:edited: False
    cf1:spoiler: False
    cf2:name: t3_1cbulhc
    cf2:sr_title: Turkish Airline flight cancelled (Malaysia -> Turkey -> Spain). Got a new flight but arrived in Spain a day later. Can I get any compensation?
    cf2:url: https://www.reddit.com/r/travel/comments/1cbulhc/turkish_airline_flight_cancelled_malaysia_turkey/
    cf3:created_utc: 1713953831.0
    cf3:num_comments: 1
    cf3:score: 1
Row Key: 008451a05e1e7aa32c75119df950d405265e0904
    cf1:edited: False
    cf1:spoiler: False
    cf2:name: t3_1cdemdr
    cf2:sr_title: Schengen visitor visa to see family member?
    cf2:url: https://www.reddit.com/r/travel/comments/1cdemdr/schengen_visitor_visa_to_see_family_member/
    cf3:created_utc: 1714113720.0
    cf3:num_comments: 9
    cf3:score: 0
Row Key: 00a6915f2bd395a55fb85aed647039e4136e35cb
    cf1:edited: False
    cf1:spoiler: False
    cf2:name: t3_1c9tcoh
    cf2:sr_title:

In [4]:
connection = happybase.Connection(port=9090)
hbase_table = connection.table('redditTravel')

df = pd.DataFrame()                    

edited, spoiler, url, name, title, created_utc, num_comments, score = [], [], [], [], [], [], [], []

# function to convert string to boolean
def str_to_bool(s: int) -> bool:
    return (lambda x: True if x == "True" else False) (s)

# function to convert integer utc to timestamp
def convert_timestamp(s: int) -> datetime:
    utc_datetime = datetime.datetime.utcfromtimestamp(s)
    timestamp_str = utc_datetime.strftime('%Y-%m-%d %H:%M:%S')
    return timestamp_str

for key, data in hbase_table.scan():
    edited.append(str_to_bool(data[b'cf1:edited'].decode('utf-8')))
    spoiler.append(str_to_bool(data[b'cf1:spoiler'].decode('utf-8')))
    url.append(data[b'cf2:url'].decode('utf-8'))
    name.append(data[b'cf2:name'].decode('utf-8'))
    title.append(data[b'cf2:sr_title'].decode('utf-8'))
    created_utc.append(convert_timestamp(int(float(data[b'cf3:created_utc'].decode('utf-8')))))
    num_comments.append(int(data[b'cf3:num_comments'].decode('utf-8')))
    score.append(int(data[b'cf3:score'].decode('utf-8')))


# add list to dataframe
df['edited'], df['spoiler'], df['url'], df['name'], df['title'], df['created_utc'], df['num_comments'], df['score'] = edited, spoiler, url, name, title, created_utc, num_comments, score
del edited, spoiler, url, name, title, created_utc, num_comments, score

df.head()

,edited,spoiler,url,name,title,created_utc,num_comments,score
0,False,False,https://www.reddit.com/r/travel/comments/1cbul...,t3_1cbulhc,Turkish Airline flight cancelled (Malaysia -> ...,2024-04-24 10:17:11,1,1
1,False,False,https://www.reddit.com/r/travel/comments/1cdem...,t3_1cdemdr,Schengen visitor visa to see family member?,2024-04-26 06:42:00,9,0
2,False,False,https://www.reddit.com/r/travel/comments/1c9tc...,t3_1c9tcoh,Ideas for a 10-12 trip in July from the US?,2024-04-21 21:32:34,4,0
3,False,False,https://www.reddit.com/r/travel/comments/1cazj...,t3_1cazj98,Prada duty free in Narita airport said my perf...,2024-04-23 08:37:23,9,0
4,False,False,https://www.reddit.com/r/travel/comments/1cbpe...,t3_1cbpewl,Vietnam evisa name in reverse order,2024-04-24 04:28:12,1,0


In [5]:
# Check if df is a Pandas DataFrame
if isinstance(df, pd.DataFrame):
    print("This is a Pandas DataFrame")

# Check if df is a Spark DataFrame
if isinstance(df, SparkDataFrame):
    print("This is a Spark DataFrame")

This is a Pandas DataFrame


In [6]:
# convert to spark dataframe (due to spark 3.3.2 , we need to downgrade pandas to 1.5.3)
spark_df = spark.createDataFrame(df)

In [7]:
# Check if df is a Pandas DataFrame
if isinstance(spark_df, pd.DataFrame):
    print("This is a Pandas DataFrame")

# Check if df is a Spark DataFrame
if isinstance(spark_df, SparkDataFrame):
    print("This is a Spark DataFrame")

This is a Spark DataFrame


In [8]:
# show table
spark_df.show(5)


[Stage 0:>                                                          (0 + 1) / 1]



+------+-------+--------------------+----------+--------------------+-------------------+------------+-----+
|edited|spoiler|                 url|      name|               title|        created_utc|num_comments|score|
+------+-------+--------------------+----------+--------------------+-------------------+------------+-----+
| false|  false|https://www.reddi...|t3_1cbulhc|Turkish Airline f...|2024-04-24 10:17:11|           1|    1|
| false|  false|https://www.reddi...|t3_1cdemdr|Schengen visitor ...|2024-04-26 06:42:00|           9|    0|
| false|  false|https://www.reddi...|t3_1c9tcoh|Ideas for a 10-12...|2024-04-21 21:32:34|           4|    0|
| false|  false|https://www.reddi...|t3_1cazj98|Prada duty free i...|2024-04-23 08:37:23|           9|    0|
| false|  false|https://www.reddi...|t3_1cbpewl|Vietnam evisa nam...|2024-04-24 04:28:12|           1|    0|
+------+-------+--------------------+----------+--------------------+-------------------+------------+-----+
only showing top 5 

In [9]:
# show schema
spark_df.printSchema()

root
 |-- edited: boolean (nullable = true)
 |-- spoiler: boolean (nullable = true)
 |-- url: string (nullable = true)
 |-- name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- num_comments: long (nullable = true)
 |-- score: long (nullable = true)



In [10]:
# get the number of rows
spark_df.count()

992

In [11]:
spark_df.show(20)

+------+-------+--------------------+----------+--------------------+-------------------+------------+-----+
|edited|spoiler|                 url|      name|               title|        created_utc|num_comments|score|
+------+-------+--------------------+----------+--------------------+-------------------+------------+-----+
| false|  false|https://www.reddi...|t3_1cbulhc|Turkish Airline f...|2024-04-24 10:17:11|           1|    1|
| false|  false|https://www.reddi...|t3_1cdemdr|Schengen visitor ...|2024-04-26 06:42:00|           9|    0|
| false|  false|https://www.reddi...|t3_1c9tcoh|Ideas for a 10-12...|2024-04-21 21:32:34|           4|    0|
| false|  false|https://www.reddi...|t3_1cazj98|Prada duty free i...|2024-04-23 08:37:23|           9|    0|
| false|  false|https://www.reddi...|t3_1cbpewl|Vietnam evisa nam...|2024-04-24 04:28:12|           1|    0|
| false|  false|https://www.reddi...|t3_1c8kf43|Cheap and interes...|2024-04-20 07:52:20|         305|  238|
| false|  false|htt

In [12]:
# get the title column
spark_df.select('title').show(n=10, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                         |
+----------------------------------------------------------------------------------------------------------------------------------------------+
|Turkish Airline flight cancelled (Malaysia -> Turkey -> Spain). Got a new flight but arrived in Spain a day later. Can I get any compensation?|
|Schengen visitor visa to see family member?                                                                                                   |
|Ideas for a 10-12 trip in July from the US?                                                                                                   |
|Prada duty free in Narita airport said my perfume could be confiscated - please help!!                                           

# Preprocessing (SparkML)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, RegexTokenizer, SQLTransformer
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import string
import json

# Function to load country codes from a JSON file
def load_country_codes():
    with open('country_codes.json', 'r') as file:
        countries = json.load(file)
    return {country['alpha2']: country['name'] for country in countries}

country_codes = load_country_codes()

# Define UDF for replacing country codes with names
def replace_country_codes(text, country_dict):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    return ' '.join([country_dict.get(word.upper(), word) for word in words])

replace_codes_udf = udf(lambda text: replace_country_codes(text, country_codes), StringType())

# SQL Transformers for various preprocessing steps
sql_transformer_lower = SQLTransformer(
    statement="SELECT *, lower(title) as lower_title FROM __THIS__")

sql_transformer_numbers = SQLTransformer(
    statement="SELECT *, regexp_replace(lower_title, '\\\\d+', '') as clean_title FROM __THIS__")

# Regex Tokenizer and StopWords Remover for text tokenization and cleanup
regex_tokenizer = RegexTokenizer(inputCol="clean_title", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Join tokens back into a single string
sql_transformer_join = SQLTransformer(
    statement="SELECT *, concat_ws(' ', filtered_words) as final_text FROM __THIS__")

# Define the pipeline stages
pipeline = Pipeline(stages=[
    sql_transformer_lower,
    sql_transformer_numbers,
    regex_tokenizer,
    remover,
    sql_transformer_join
])

# Load DataFrame
# Assuming spark_df is defined
spark_df = spark_df.withColumn("preprocessed_title", replace_codes_udf(col("title")))
model = pipeline.fit(spark_df)
transformed_df = model.transform(spark_df)

transformed_df.select('title', 'preprocessed_title').show(n=5, truncate=False)

# Preprocessing (NLTK)

In [24]:
import string
import json
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

def load_country_codes(json_filepath):
    with open(json_filepath, 'r') as file:
        countries = json.load(file)
    country_codes = {country['alpha2']: country['name'] for country in countries}
    country_codes.update({country['alpha3']: country['name'] for country in countries})
    return country_codes

country_codes = load_country_codes('country_codes.json')

def convert_country_code_to_name(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    converted_words = [country_codes.get(word.upper(), word) for word in words]
    return ' '.join(converted_words)

def preprocess_text(text):
    try:
        # Convert country codes
        text = convert_country_code_to_name(text)

        # Convert text to lowercase
        text = text.lower()
        
        # Remove numbers
        text = re.sub(r'\d+', '', text)

        # Tokenize text
        tokens = word_tokenize(text)

        # Remove stopwords (assuming English stopwords)
        stop_words = set(stopwords.words('english'))
        tokens = [token for token in tokens if token not in stop_words]

        # Join tokens into a single string
        return ' '.join(tokens)
    except Exception as e:
        print("Error in preprocessing:", e)
        return None
    
udf_preprocess_text = udf(preprocess_text, StringType())

In [26]:
spark_df = spark_df.withColumn('preprocessed_title', udf_preprocess_text('title'))

In [27]:
spark_df.select('title', 'preprocessed_title').show(n=5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|title                                                                                                                                         |preprocessed_title                                                                                                   |
+----------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+
|Turkish Airline flight cancelled (Malaysia -> Turkey -> Spain). Got a new flight but arrived in Spain a day later. Can I get any compensation?|turkish airline flight cancelled malaysia turkey spain got new flig

In [29]:
# Select the "title" column from the PySpark DataFrame
title_df = spark_df.select('preprocessed_title')

# Save the selected column to MongoDB
title_df.write.format("mongo") \
    .mode("overwrite") \
    .option("uri", "mongodb://127.0.0.1/data_engineering.preprocessed.coll") \
    .option("collection", "title_collection") \
    .save()